<div style="display: flex; background-color: RGB(203, 192, 225);" >
<h1 style="margin: auto; padding: 30px; ">Notebook extracting trends LDA</h1>
</div>

Objectif : Extracting the main topics from your dataset using LDA

In [12]:
# import des librairies
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# preprocessing data
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# pip install gensim
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

from sklearn.model_selection import train_test_split

In [13]:
# import des données
df = pd.read_csv('data/gsearch_jobs.csv')
df.head()

,Unnamed: 0,index,title,company_name,location,via,description,extensions,job_id,thumbnail,...,commute_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,0,Data Analyst (Risk Adjustment Consulting Resea...,"Cambia Health Solutions, Inc",United States,via Datafloq,Are you looking for a new job? Check out this ...,"['3 hours ago', 'Full-time', 'No degree mentio...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgKFJpc2sgQW...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,1,1,DATA ANALYST II,Lumen,United States,via ComputerJobs.com,About Lumen\nLumen is guided by our belief tha...,"['17 hours ago', 'Full-time', 'No degree menti...",eyJqb2JfdGl0bGUiOiJEQVRBIEFOQUxZU1QgSUkiLCJodG...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'sql', 'powerpoint', 'power_bi', 'sh..."
2,2,2,Data Analyst - Swisslog,Swisslog,United States,via Swisslog,"Data Analyst Mason, Ohio With guidance from se...","['4 hours ago', 'Full-time', 'Health insurance...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgLSBTd2lzc2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['python', 'r', 'sql', 'powerpoint', 'word', '..."
3,3,3,Data Analyst - Secret clearance - Remote Remot...,General Dynamics Information Technology,Anywhere,via Clearance Jobs,REQ#: RQ135670 Travel Required: None Public Tr...,"['11 hours ago', 'Work from home', 'Full-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgLSBTZWNyZX...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['t-sql', 'pl/sql', 'sql']"
4,4,4,Collections Data Analyst (921071),Purpose Financial,United States,via Jobs At Purpose Financial / Advance Americ...,"Address : 135 N Church Street, Spartanburg, So...","['20 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJDb2xsZWN0aW9ucyBEYXRhIEFuYW...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['python', 'r', 'sas', 'sql']"


<div style="background-color: RGB(165, 195, 186);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">EDA</h2>
</div>

### NaN

In [14]:
def count_nan(df):
    
    nan_counts = df.isna().sum() # compte le nombre de NaN pour chaque colonne
    total_counts = len(df) # compte le nombre total de données dans le dataframe
    nan_percentages = (nan_counts / total_counts) * 100 # calcule le pourcentage de NaN pour chaque colonne
    result_df = pd.concat([nan_counts, nan_percentages], axis=1) # combine les deux séries en un dataframe
    result_df.columns = ['NaN Count', 'NaN Percentage'] # renomme les colonnes du nouveau dataframe
    return result_df

In [15]:
df_NaN = count_nan(df)
df_NaN = df_NaN.sort_values(by = ['NaN Count'], ascending = False)
# df_NaN = df_NaN.loc[df_NaN['NaN Count'] != 0]
df_NaN

,NaN Count,NaN Percentage
commute_time,17977,100.000000
salary_yearly,16525,91.923013
salary_hourly,15966,88.813484
salary_max,14720,81.882405
salary_min,14720,81.882405
salary,14508,80.703121
salary_standardized,14508,80.703121
salary_avg,14508,80.703121
salary_rate,14508,80.703121
salary_pay,14508,80.703121


In [16]:
# supprimer les colonnes qui ont trop de NaN
def no_NaN(df, treshold):
    
    nan_counts = df.isna().sum() # compte le nombre de NaN pour chaque colonne
    total_counts = len(df) # compte le nombre total de données dans le dataframe
    nan_percentages = (nan_counts / total_counts) * 100 # calcule le pourcentage de NaN pour chaque colonne
    nan_treshold = nan_percentages[nan_percentages.values < treshold]
    
    return df[nan_treshold.index]

In [17]:
# df version 2
df_v2 = no_NaN(df, 50)
df_v2 = df_v2.dropna()
df_v2.isnull().sum()

Unnamed: 0            0
index                 0
title                 0
company_name          0
location              0
via                   0
description           0
extensions            0
job_id                0
posted_at             0
schedule_type         0
search_term           0
date_time             0
search_location       0
description_tokens    0
dtype: int64

### Preprocess

In [18]:
# Split the dataset into train and test sets
df_train, df_test = train_test_split(df_v2, test_size=0.2, random_state=42)


<div style="background-color: RGB(165, 195, 186);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Extracting Topics using LDA in Python</h2>
</div>

In [19]:
# Récupérer la colonne "description" sous forme de liste
description_list = df_train["description"].tolist()

In [20]:
stemmer = SnowballStemmer("english")

In [21]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [22]:
# Créer une liste pour stocker les résultats
processed_text = []

# Appliquer la fonction lemmatize_stemming à chaque élément de la liste description_list
for text in description_list:
    processed_text.append(preprocess(text))

In [23]:
'''
Preview 'processed_docs'
'''
print(processed_text[0])

['need', 'help', 'data', 'analysi', 'qualit', 'research', 'nvivo']


In [24]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_text)

In [25]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

0 analysi
1 data
2 help
3 need
4 nvivo
5 qualit
6 research
7 confirmatori
8 determin
9 excel
10 indic
11 relationship
12 sensit
13 sheet
14 specif
15 statstic
16 accomplish
17 accord
18 account
19 accuraci
20 achiev


- less than no_below documents (absolute number) or
- more than no_above documents (fraction of total corpus size, not absolute number).
- after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [ ]:
'''
OPTIONAL STEP
Remove very rare and ver common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_text]

In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))


<div style="background-color: RGB(165, 195, 186);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">Running LDA using Bag of Words</h2>
</div>

In [ ]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

## Classification of the topics

Quelques suggestions de noms pour chaque topic basées sur les mots et les poids correspondants : 

- Topic 0: Carrières dans la publication et l'édition
- Topic 1: Assurance et remboursement médical
- Topic 2: Marketing et publicité en ligne
- Topic 3: Éducation et études universitaires
- Topic 4: Recherche médicale et scientifique
- Topic 5: Logistique et transport
- Topic 6: Informatique et résolution de problèmes
- Topic 7: Finance et gestion des actifs

## Vizualisation

In [ ]:
import plotly.express as px

In [ ]:
# Créer un dataframe avec les informations des topics
topics_df = pd.DataFrame({
    'Topic': ["Carrières dans la publication et l'édition", 'Assurance et remboursement médical', 
              'Marketing et publicité en ligne', 'Éducation et études universitaires', 
              'Recherche médicale et scientifique', 'Logistique et transport',
              'Informatique et résolution de problèmes', 'Finance et gestion des actifs'],
    'Words': [
        'jone, edward, fortun, firm, branch, rank, magazin, publish, rat, obtain',
        'claim, centen, clinic, purchas, reimburs, elev, stock, pharmacie, root, tuition',
        'brand, media, websit, campaign, event, looker, consum, love, tech, tulsa',
        'univers, student, school, studi, survey, colleg, check, academ, institut, date',
        'clinic, suppli, machin, chain, patient, walmart, predict, scientist, algorithm, energi',
        'resum, agenc, capac, day, transport, send, forward, ongo, orchestr, readi',
        'oracl, map, file, troubleshoot, server, agil, resolv, flow, vendor, travel',
        'vaccin, bank, covid, audit, citi, asset, investig, architectur, azur, legal'
    ],
    'Weight': [
        [0.059, 0.052, 0.047, 0.045, 0.040, 0.033, 0.026, 0.025, 0.025, 0.024],
        [0.021, 0.016, 0.013, 0.012, 0.012, 0.012, 0.012, 0.012, 0.011, 0.010],
        [0.007, 0.007, 0.006, 0.006, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005],
        [0.016, 0.014, 0.009, 0.008, 0.008, 0.007, 0.007, 0.006, 0.006, 0.005],
        [0.014, 0.010, 0.010, 0.009, 0.009, 0.008, 0.008, 0.008, 0.006, 0.006],
        [0.031, 0.021, 0.017, 0.017, 0.016, 0.015, 0.013, 0.013, 0.013, 0.013],
        [0.008, 0.007, 0.006, 0.006, 0.006, 0.005, 0.005, 0.005, 0.004, 0.004],
        [0.010, 0.009, 0.006, 0.006, 0.005, 0.005, 0.005, 0.005, 0.005, 0.004]
    ]
})

In [ ]:
topics_df

### Exporter le topics_df

In [ ]:
topics_df = topics_df.to_csv("data/topics_df.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

In [ ]:
# Create a list of all words
all_words = [word for words_list in topics_df['Words'] for word in words_list.split(", ")]

# Create a list of all weights
all_weights = [weight for weight_list in topics_df['Weight'] for weight in weight_list]

# Create a list of topics for each word
topic_labels = []
for i, words_list in enumerate(topics_df['Words']):
    num_words = len(words_list.split(", "))
    topic_labels.extend([topics_df['Topic'][i]] * num_words)
    
# Create hover text with words and topics
hover_text = [f"Word: {word}<br>Topic: {topic}" for word, topic in zip(all_words, topic_labels)]

# Create a Scatter3d trace with the data
scatter3d_trace = go.Scatter3d(
    x=list(range(len(all_words))),
    y=[0] * len(all_words),
    z=all_weights,
    text=all_words,
    hovertemplate="%{text}<br><br>%{hovertext}",
    hovertext=hover_text,
    mode='markers',
    marker=dict(
        size=[weight * 1000 for weight in all_weights],
        opacity=0.8
    )
)


# Create a Scatter3d trace with the data
scatter3d_trace = go.Scatter3d(
    x=list(range(len(all_words))),
    y=[0] * len(all_words),
    z=all_weights,
    text=all_words,
    hovertext=topic_labels,  # Assign the topic labels as hovertext
    mode='markers',
    marker=dict(
        size=[weight * 1000 for weight in all_weights],
        opacity=0.8
    )
)
    
# Create a Figure object and add the trace
fig = go.Figure(data=[scatter3d_trace])

# Configure the layout
fig.update_layout(
    title='Scatter Plot 3D - Mots et poids',
    scene=dict(
        xaxis=dict(
            title='Mots',
            tickfont=dict(color='white'),
            title_font=dict(color='white')
        ),
        yaxis=dict(
            title='Y',
            tickfont=dict(color='white'),
            title_font=dict(color='white')
        ),
        zaxis=dict(
            title='Poids',
            tickfont=dict(color='white'),
            title_font=dict(color='white')
        ),
        bgcolor='black'
    ),
    hovermode='closest',
    plot_bgcolor='black'
)    
  
# Show the interactive figure
fig.show()